In [235]:
import numpy as np, random, operator
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from itertools import starmap
from numba import njit
import folium
import math
import string
from folium.plugins import TimestampedGeoJson
from folium.plugins import HeatMap
#from shapely import wkt, geometry
import json
from pprint import pprint
import sys
import datetime

In [236]:
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [237]:
from openrouteservice import client, places

api_key = '5b3ce3597851110001cf6248e00f68cbe8784f68a7e1760dd297b7e1'

clnt = client.Client(key=api_key)

In [238]:
data = pd.read_csv(r'C:\Users\502\Desktop\11월18일(m).csv',encoding='cp949')
x = data.x_coord
y = data.y_coord
cityList = [(b,a) for a,b in zip(x,y)]
cityList_ = [(127.2498965,37.3525179)]+[(b,a) for a,b in zip(y,x)]
city_names = [i for i in data.node_name]
final_box = [0] + [i for i in data.final_box]
final_cost = [0] + [i for i in data.save_cost]
x_coord = [127.2498965] + [i for i in x]
y_coord = [37.3525179] + [i for i in y]
address = ['태산팩'] + [i for i in data.address]
data['node_coord']=[(b,a) for a,b in zip(x,y)]

In [239]:
data

,node_name,date,product,address,final_box,save_cost,y_coord,x_coord,node_coord
0,훼미리유통,2019-11-18,커피컵8온스(블랙문자)/커피컵뚜껑8온스-백색,경기도 광주시 탄벌길 128,3,9500,37.421270,127.234540,"(37.42126967, 127.23454029999999)"
1,종로떡방(미금),2019-11-18,태화30호,경기도 성남시 분당구 구미동 90번길 4,4,12000,37.349715,127.111735,"(37.349715100000004, 127.1117354)"
2,한촌탕반명가(정춘호),2019-11-18,HG713호(죽용기-대)세트,경기도 용인시 수지구 현암로 140 1층,10,35000,37.331499,127.124763,"(37.33149885, 127.12476310000001)"
3,진수성찬,2019-11-18,61비닐(청색)/비닐봉투대(검정)/태양 36호,경기도 용인시 수지구 대지로 43,5,13000,37.328612,127.114193,"(37.32861177, 127.11419340000002)"
4,김문식,2019-11-18,초밥도시락1인용,경기도 고양시 일산동구 장백로 61,1,3000,37.643051,126.785664,"(37.64305135, 126.7856642)"
5,떡깨비(광명점),2019-11-18,실링기계MA-1(몰드포함),경기도 광명시 광명로865번길 2,1,4000,37.476524,126.852687,"(37.47652384, 126.85268729999999)"
6,와이에스패밀리(광명),2019-11-18,GP-120-소(4칸)set,경기도 광명시 오리로 645 (하안동) 1층,4,12000,37.458786,126.877333,"(37.45878629999999, 126.87733259999999)"
7,광명홍어(광명),2019-11-18,GD-003검정(BK)set/GD-004검정(BK)set,경기도 광명시 일직로99번길 21,8,26000,37.421381,126.890874,"(37.42138119, 126.8908738)"
8,엄마소반,2019-11-18,HG303호(21호)백색/HG305(3-5호)백색,경기도 광주시 오포읍 문현로 1-9,2,8000,37.348722,127.213431,"(37.34872242, 127.21343119999999)"
9,아는언니,2019-11-18,호프용기(1L)-111개,경기도 광주시 오포읍 양벌로 303-12,1,5000,37.371413,127.248559,"(37.37141348, 127.2485587)"


In [240]:
# n_data => 인접지역 제거
#n_data=data[~(data['address'].str.contains('수원')) & ~(data['address'].str.contains('용인'))&~(data['address'].str.contains('광주'))&~(data['address'].str.contains('성남'))]
n_data = data.copy()
n_data.sort_values(by='save_cost',ascending=False,inplace=True)
n_data.reset_index(drop=True,inplace=True)
nodeList = [(37.3525179,127.2498965)]+n_data['node_coord'].values.tolist()
nodeList_ = [(127.2498965,37.3525179)]+[(a,b) for a,b in zip(n_data['x_coord'],n_data['y_coord'])] # api전용
node_name = ['태산팩']+n_data['node_name'].values.tolist()
node_box = [0] + n_data['final_box'].values.tolist()
f_data = n_data.copy()

In [241]:
from openrouteservice import distance_matrix

request = {'locations': nodeList_,
           'profile': 'driving-car',
           'metrics': ['duration']}
    
distance_matrix = clnt.distance_matrix(**request)
print("Calculated {}x{} routes.".format(len(distance_matrix['durations']),len(distance_matrix['durations'][0])))

Calculated 50x50 routes.


In [242]:
def node_renewal(n_data):
    m1 = 0
    m2 = 1
    truck1 = []
    cap1 = []
    cost1= []
    name1 = []
    truck1.append(n_data['node_coord'][m1])
    sum_box1 = n_data['final_box'][m1]
    name1.append(n_data['node_name'][m1])
    cap1.append(n_data['final_box'][m1])
    cost1.append(n_data['save_cost'][m1])
    n_data.drop(m1,inplace=True)

    truck2 = []
    name2 = []
    cap2 = []
    cost2 = []
    truck2.append(n_data['node_coord'][m2])
    sum_box2 = n_data['final_box'][m2]
    name2.append(n_data['node_name'][m2])
    cap2.append(n_data['final_box'][m2])
    cost2.append(n_data['save_cost'][m2])
    n_data.drop(m2,inplace=True)

    x1_i = []
    y1_i = []
    for i in n_data.index:
        x1_i.append(distance_matrix['durations'][0][i+1])
    for i in n_data.index:
        y1_i.append(distance_matrix['durations'][i+1][m1+1])

    n_data['x1_i'] = x1_i
    n_data['y1_i'] = y1_i
    n_data['x1+y1'] = n_data['x1_i'] + n_data['y1_i'] 
    new_d1 = n_data['x1+y1'].idxmin()
    truck1.append(n_data['node_coord'][new_d1])
    sum_box1 = sum_box1 + n_data['final_box'][new_d1]
    name1.append(n_data['node_name'][new_d1])
    cap1.append(n_data['final_box'][new_d1])
    cost1.append(n_data['save_cost'][new_d1])
    n_data.drop(new_d1,inplace=True)

    x2_i = []
    y2_i = []
    for i in n_data.index:
        x2_i.append(distance_matrix['durations'][0][i+1])
    for i in n_data.index:
        y2_i.append(distance_matrix['durations'][i+1][m2+1])

    n_data['x2_i'] = x2_i
    n_data['y2_i'] = y2_i
    n_data['x2+y2'] = n_data['x2_i'] + n_data['y2_i'] 
    new_d2 = n_data['x1+y1'].idxmin()
    truck2.append(n_data['node_coord'][new_d2])
    sum_box2 = sum_box2 + n_data['final_box'][new_d2]
    name2.append(n_data['node_name'][new_d2])
    cap2.append(n_data['final_box'][new_d2])
    cost2.append(n_data['save_cost'][new_d2])
    n_data.drop(new_d2,inplace=True)

    nn_data = n_data.copy()
    
    for i in range(len(n_data)):
    #nn_data.drop(new_d1,inplace=True)
    #nn_data.drop(new_d2,inplace=True)
        x1_i = []
        y1_i = []
        for i in nn_data.index:
            x1_i.append(distance_matrix['durations'][new_d1+1][i+1])
        for i in nn_data.index:
            y1_i.append(distance_matrix['durations'][i+1][m1+1])
        nn_data['x1_i'] = x1_i
        nn_data['y1_i'] = y1_i
        nn_data['x1+y1'] = nn_data['x1_i'] + nn_data['y1_i']
        new_d1 = nn_data['x1+y1'].idxmin()
        truck1.append(nn_data['node_coord'][new_d1])
        sum_box1 = sum_box1 + nn_data['final_box'][new_d1]
        name1.append(nn_data['node_name'][new_d1])
        cap1.append(n_data['final_box'][new_d1])
        cost1.append(n_data['save_cost'][new_d1])
        if sum_box1 > 60:
            sum_box1 = sum_box1 - n_data['final_box'][new_d1]
            del truck1[-1]
            del name1[-1]
            del cap1[-1]
            del cost1[-1] 
        else:
            nn_data.drop(new_d1,inplace=True)
    
    
        
        x2_i = []
        y2_i = []
        for i in nn_data.index:
            x2_i.append(distance_matrix['durations'][new_d2+1][i+1])
        for i in nn_data.index:
            y2_i.append(distance_matrix['durations'][i+1][m2+1])
        nn_data['x2_i'] = x2_i
        nn_data['y2_i'] = y2_i
        nn_data['x2+y2'] = nn_data['x2_i'] + nn_data['y2_i'] 
        new_d2 = nn_data['x2+y2'].idxmin()
        truck2.append(nn_data['node_coord'][new_d2])
        sum_box2 = sum_box2 + nn_data['final_box'][new_d2]
        name2.append(nn_data['node_name'][new_d2])
        cap2.append(n_data['final_box'][new_d2])
        cost2.append(n_data['save_cost'][new_d2])

        if sum_box2 > 60:
            sum_box2 = sum_box2 - n_data['final_box'][new_d2]
            del truck2[-1]
            del name2[-1]
            del cap2[-1]
            del cost2[-1] 
        else:
            nn_data.drop(new_d2,inplace=True)


        if len(nn_data)==0:
            break

    return truck1, truck2, sum_box1, sum_box2, cap1, cap2, cost1, cost2, name1, name2

In [243]:
result=node_renewal(n_data)

In [244]:
name1 = result[8]
location1 = result[0]
a = [nodeList_[nodeList.index(i)] for i in location1]
demands1 = result[4]
cost1 = result[6]

In [245]:
name2= result[9]
location2= result[1]
ab = [nodeList_[nodeList.index(i)] for i in location2]
demands2 = result[5]
cost2 = result[7]

In [246]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Locations in block units
    data['name'] = name1
    data['locations'] = a
    data['num_vehicles'] = 1
    data['demands'] = demands1
    data['vehicle_capacities'] = [60]
    data['depot'] = 0
    data['cost']=cost1
    return data

In [247]:
def compute_cost_matrix(locations):
    """Creates callback to return distance between points."""
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = data['cost'][to_counter]
    return distances
# save한 비용

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(distance_matrix['durations'][from_node][to_node])


def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

result =[]

def draw_solution(data, manager=None, routing=None, assignment=None):
    locations = data['locations']
    
    if manager is not None and routing is not None and assignment is not None:
        
        totalnodesolution = []
        for vehicle_id in range(data['num_vehicles']):
            route_cost = 0
            route_load = 0
            node_solution = []
            arc_solution = []
            save_cost = 0
            index = routing.Start(vehicle_id)
            while not routing.IsEnd(index):
                i = manager.IndexToNode(index)
                node_solution.append(i)
                route_load += data['demands'][i]
                save_cost += data['cost'][i]
                previous_index = index
                index = assignment.Value(routing.NextVar(index))
                j = manager.IndexToNode(index)
                arc_solution.append((i,j))
                route_cost += routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id)
            route_cost += routing.GetArcCostForVehicle(
                        i, 0, vehicle_id)   # 마지막 노드에서 태산팩까지의 거리를 더하는 과정    

            print(f"Route: {node_solution}")
            print(f"Eventual distance of truck1: {route_cost}")
            print(f"Route load: {route_load}")
            print(f"save cost: {save_cost}")
            totalnodesolution.append(node_solution)
            result.append(route_cost)
            
    return (totalnodesolution)

    

In [248]:
data =create_data_model()

In [249]:
data 

{'name': ['우수한형제들(본점)',
  '(주)사람과교육(스쿨서플라이)',
  '소라떡볶이',
  '덕보갈비탕',
  '비엘포장(유통)',
  '바른치킨용인동백점(손유미)',
  '잠실감자탕',
  '순이네빈대떡',
  '훼미리유통',
  '샤브담 여의대방로(본점)',
  '용브라더키친(정득규)',
  '돈까스잔치동빙고동점(에프씨천상)',
  '광명홍어(광명)',
  '와이에스패밀리(광명)',
  '떡깨비(광명점)',
  '새우식탁(서동우)'],
 'locations': [(127.03058119999999, 37.28105653),
  (127.15711029999999, 37.3313419),
  (127.08803219999999, 37.31939948),
  (127.03472579999999, 37.262769),
  (127.1366831, 37.26757192),
  (127.15399920000002, 37.27768222),
  (127.16127150000001, 37.26111673),
  (127.2320922, 37.37003209),
  (127.23454029999999, 37.42126967),
  (127.02006159999999, 37.50345553),
  (127.03525020000001, 37.50889899),
  (126.9964348, 37.52125324),
  (126.8908738, 37.42138119),
  (126.87733259999999, 37.45878629999999),
  (126.85268729999999, 37.47652384),
  (126.8045953, 37.48649111)],
 'num_vehicles': 1,
 'demands': [18, 1, 3, 1, 8, 1, 6, 1, 3, 1, 1, 2, 8, 4, 1, 1],
 'vehicle_capacities': [60],
 'depot': 0,
 'cost': [60000,
  4000,
  9500,
  2500,
  32

In [250]:
data['demands'] = [0]+ data['demands'] 
data['name']=['태산팩']+data['name'] 
data['locations']=[(127.2498965,37.3525179)]+data['locations']
data['cost'] = [0]+data['cost'] 
a.insert(0,(127.2498965,37.3525179))
location1.insert(0,(37.3525179,127.2498965))

In [251]:
from openrouteservice import distance_matrix

request = {'locations': data['locations'],
           'profile': 'driving-car',
           'metrics': ['duration']}
    
distance_matrix = clnt.distance_matrix(**request)
print("Calculated {}x{} routes.".format(len(distance_matrix['durations']),len(distance_matrix['durations'][0])))

Calculated 17x17 routes.


In [252]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [253]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data['locations']),
    data['num_vehicles'], 
    data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)#callback 2?

# Register the demand callback
demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback) # unary 일원 ,_한줄짜리라고 추측됨 // 첫번쨰? callback1?


# Add dimension for the demand
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    144000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)



# Minimize the maximum travel distance
distance_dimension.SetGlobalSpanCostCoefficient(35)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
assignment = routing.SolveWithParameters(search_parameters)
#search_parameters.time_limit.seconds = 60 
# Print solution on console.

if assignment:
    list_=draw_solution(data, manager, routing, assignment)
    for i in range(len(list_)):
        route_sol = list_[i]
        route_visit_names = [data['name'][i] for i in route_sol]
        print(f'Solution: {" -> ".join(route_visit_names)}','->태산팩 ')
        #print(list_)
else : 
    print('::')

Route: [0, 2, 3, 6, 7, 5, 4, 1, 13, 14, 15, 16, 12, 10, 11, 9, 8]
Eventual distance of truck1: 11179
Route load: 60
save cost: 201750
Solution: 태산팩 -> (주)사람과교육(스쿨서플라이) -> 소라떡볶이 -> 바른치킨용인동백점(손유미) -> 잠실감자탕 -> 비엘포장(유통) -> 덕보갈비탕 -> 우수한형제들(본점) -> 광명홍어(광명) -> 와이에스패밀리(광명) -> 떡깨비(광명점) -> 새우식탁(서동우) -> 돈까스잔치동빙고동점(에프씨천상) -> 샤브담 여의대방로(본점) -> 용브라더키친(정득규) -> 훼미리유통 -> 순이네빈대떡 ->태산팩 


In [254]:
# 폴리곤 + 마커 생성 ( 마커는 태산팩을 포함한(+1)원래의 좌표에 생성 , 폴리곤은 임의로 생성 )

m = folium.Map(location=[37.3525179,127.2498965], zoom_start=10)

for feat in location1:
    folium.CircleMarker(feat,radius=2,color='red').add_to(m)

#for feat in result_lst[1]:
    #folium.CircleMarker(feat,radius=2,color='blue').add_to(m)    
    
#for feat in result_lst[2]:
    #flium.CircleMarker(feat,radius=2,color='black').add_to(m)    
    
folium.Marker(
          location=[37.3525179,127.2498965],
          popup='태산팩',
          icon=folium.Icon(color='red',icon='star')).add_to(m)

m

In [255]:
# 두 경로를 직선으로 표현 (거리는 차량거리로 계산한거임)
optimal_coords =[]
a =[(data['locations'][i]) for i in list_[0]]
a = a+ [(127.2498965,37.3525179)]

optimal_coords.append(a)
optimal_coords
for i in range(len(optimal_coords)):
    route_sol_polyline = {
    'type': 'LineString',
    'coordinates': optimal_coords[i]
    }

    def style_function(color):
        return lambda feature: dict(color=color,
                                  weight=3,
                                  opacity=1)
    if i == 0:      
        folium.features.GeoJson(data=route_sol_polyline,
                                style_function=style_function('red'),
                               overlay=True).add_to(m)
    else :
        folium.features.GeoJson(data=route_sol_polyline,
                                style_function=style_function('blue'),
                               overlay=True).add_to(m)
m

In [256]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Locations in block units
    data['name'] = name2
    data['locations'] = ab
    data['num_vehicles'] = 1
    data['demands'] = demands2
    data['vehicle_capacities'] = [60]
    data['depot'] = 0
    data['cost']=cost2
    return data

In [257]:
def compute_cost_matrix(locations):
    """Creates callback to return distance between points."""
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = data['cost'][to_counter]
    return distances
# save한 비용

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(distance_matrix['durations'][from_node][to_node])


def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

result =[]

def draw_solution(data, manager=None, routing=None, assignment=None):
    locations = data['locations']
    
    if manager is not None and routing is not None and assignment is not None:
        
        totalnodesolution = []
        for vehicle_id in range(data['num_vehicles']):
            route_cost = 0
            route_load = 0
            node_solution = []
            arc_solution = []
            save_cost = 0
            index = routing.Start(vehicle_id)
            while not routing.IsEnd(index):
                i = manager.IndexToNode(index)
                node_solution.append(i)
                route_load += data['demands'][i]
                save_cost += data['cost'][i]
                previous_index = index
                index = assignment.Value(routing.NextVar(index))
                j = manager.IndexToNode(index)
                arc_solution.append((i,j))
                route_cost += routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id)
            route_cost += routing.GetArcCostForVehicle(
                        i, 0, vehicle_id)   # 마지막 노드에서 태산팩까지의 거리를 더하는 과정    

            print(f"Route: {node_solution}")
            print(f"Eventual distance of truck2: {route_cost}")
            print(f"Route load: {route_load}")
            print(f"save cost: {save_cost}")
            totalnodesolution.append(node_solution)
            result.append(route_cost)
            
    return (totalnodesolution)

    

In [258]:
data =create_data_model()

In [259]:
data['demands'] = [0]+ data['demands'] 
data['name']=['태산팩']+data['name'] 
data['locations']=[(127.2498965,37.3525179)]+data['locations']
data['cost'] = [0]+data['cost'] 
ab.insert(0,(127.2498965,37.3525179))
location2.insert(0,(37.3525179,127.2498965))

In [260]:
from openrouteservice import distance_matrix

request = {'locations': data['locations'],
           'profile': 'driving-car',
           'metrics': ['duration']}
    
distance_matrix = clnt.distance_matrix(**request)
print("Calculated {}x{} routes.".format(len(distance_matrix['durations']),len(distance_matrix['durations'][0])))

Calculated 18x18 routes.


In [261]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data['locations']),
    data['num_vehicles'], 
    data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)#callback 2?

# Register the demand callback
demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback) # unary 일원 ,_한줄짜리라고 추측됨 // 첫번쨰? callback1?


# Add dimension for the demand
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    144000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)



# Minimize the maximum travel distance
distance_dimension.SetGlobalSpanCostCoefficient(35)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
assignment = routing.SolveWithParameters(search_parameters)
#search_parameters.time_limit.seconds = 60 
# Print solution on console.

if assignment:
    list_=draw_solution(data, manager, routing, assignment)
    for i in range(len(list_)):
        route_sol = list_[i]
        route_visit_names = [data['name'][i] for i in route_sol]
        print(f'Solution: {" -> ".join(route_visit_names)}','->태산팩 ')
        #print(list_)
else : 
    print('::')

Route: [0, 7, 1, 2, 3, 4, 5, 6, 9, 10, 11, 17, 16, 12, 13, 14, 15, 8]
Eventual distance of truck2: 8610
Route load: 59
save cost: 194250
Solution: 태산팩 -> 엄마소반 -> 한촌탕반명가(정춘호) -> 진수성찬 -> 종로떡방(미금) -> 김준희((도시락을부탁해))성남 -> 두부장수(분당서현) -> 장독대야탑역점 -> 디랩(강남점)조국 -> 윤범호(보리스짐)서울 -> 주식회사 프레시코드 -> 전주콩나루콩나물국밥중계점 -> 김일호(남양주) -> 김솜이(오빠츄러스 언니커피)구리 -> 정지수(더팝)구리 -> 김종석(용령감자탕) -> 어왕(김용진) -> 아는언니 ->태산팩 


In [262]:
for feat in location2:
    folium.CircleMarker(feat,radius=2,color='blue').add_to(m)

#for feat in result_lst[1]:
    #folium.CircleMarker(feat,radius=2,color='blue').add_to(m)    
    
#for feat in result_lst[2]:
    #flium.CircleMarker(feat,radius=2,color='black').add_to(m)    


optimal_coords =[]
a =[(data['locations'][i]) for i in list_[0]]
a =  a+ [(127.2498965,37.3525179)]

optimal_coords.append(a)
optimal_coords
for i in range(len(optimal_coords)):
    route_sol_polyline = {
    'type': 'LineString',
    'coordinates': optimal_coords[i]
    }

    def style_function(color):
        return lambda feature: dict(color=color,
                                  weight=3,
                                  opacity=1)
    if i == 0:      
        folium.features.GeoJson(data=route_sol_polyline,
                                style_function=style_function('blue'),
                               overlay=True).add_to(m)
    else :
        folium.features.GeoJson(data=route_sol_polyline,
                                style_function=style_function('green'),
                               overlay=True).add_to(m)
m

In [263]:
# 폴리곤 + 마커 생성 ( 마커는 태산팩을 포함한(+1)원래의 좌표에 생성 , 폴리곤은 임의로 생성 )

m0 = folium.Map(location=[37.3525179,127.2498965], zoom_start=10)

for feat in location1:
    folium.CircleMarker(feat,radius=2,color='red').add_to(m0)

#for feat in result_lst[1]:
    #folium.CircleMarker(feat,radius=2,color='blue').add_to(m)    
    
#for feat in result_lst[2]:
    #flium.CircleMarker(feat,radius=2,color='black').add_to(m)    
    
folium.Marker(
          location=[37.3525179,127.2498965],
          popup='태산팩',
          icon=folium.Icon(color='red',icon='star')).add_to(m0)

for feat in location2:
    folium.CircleMarker(feat,radius=2,color='blue').add_to(m0)

#for feat in result_lst[1]:
    #folium.CircleMarker(feat,radius=2,color='blue').add_to(m)    
    
#for feat in result_lst[2]:
    #flium.CircleMarker(feat,radius=2,color='black').add_to(m)    


optimal_coords =[]
a =[(data['locations'][i]) for i in list_[0]]
a =  a+ [(127.2498965,37.3525179)]

optimal_coords.append(a)
optimal_coords
for i in range(len(optimal_coords)):
    route_sol_polyline = {
    'type': 'LineString',
    'coordinates': optimal_coords[i]
    }

    def style_function(color):
        return lambda feature: dict(color=color,
                                  weight=3,
                                  opacity=1)
    if i == 0:      
        folium.features.GeoJson(data=route_sol_polyline,
                                style_function=style_function('blue'),
                               overlay=True).add_to(m0)
    else :
        folium.features.GeoJson(data=route_sol_polyline,
                                style_function=style_function('green'),
                               overlay=True).add_to(m0)
m0